In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 19/02/2025.
@author: Air.Zou
"""
# ... existing code ...
import tushare as ts
import pandas as pd
# 设置 tushare token
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()

# df = pro.fund_daily(ts_code='510310.SH', start_date='20241010', end_date='20250225')
df = pro.daily(ts_code='300059.SZ', start_date='20241010', end_date='20250225')

print(df.head())


     ts_code trade_date   open   high    low  close  pre_close  change  \
0  300059.SZ   20250221  23.65  24.52  23.34  24.35      23.57    0.78   
1  300059.SZ   20250220  23.72  23.83  23.49  23.57      23.82   -0.25   
2  300059.SZ   20250219  23.30  23.96  23.26  23.82      23.33    0.49   
3  300059.SZ   20250218  24.20  24.27  23.28  23.33      24.37   -1.04   
4  300059.SZ   20250217  24.56  24.73  24.16  24.37      24.22    0.15   

   pct_chg         vol        amount  
0   3.3093  7552708.73  1.819196e+07  
1  -1.0495  2958360.22  6.987388e+06  
2   2.1003  4209937.42  9.973690e+06  
3  -4.2675  5138122.50  1.222915e+07  
4   0.6193  4793366.70  1.172428e+07  


In [3]:
df["date"]=pd.to_datetime(df["trade_date"])
df['openinterest']=0
df.rename(columns={'amount':'volume'}, inplace=True)
df.set_index("date",inplace=True)
df.sort_index(inplace=True)
df = df[["open","high","low","close","volume","openinterest"]]
print(df.head())


             open   high    low  close        volume  openinterest
date                                                              
2024-10-10  24.10  24.56  19.92  20.55  5.197314e+07             0
2024-10-11  19.51  21.50  19.51  20.85  3.014329e+07             0
2024-10-14  20.65  21.60  18.89  21.07  3.339721e+07             0
2024-10-15  20.30  21.26  20.13  20.16  2.363889e+07             0
2024-10-16  19.64  21.07  19.64  20.38  2.288313e+07             0


In [ ]:

import matplotlib
import pandas as pd
import talib
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
import akshare as ak
import numpy as np
class MyStrategy(bt.Strategy):
    def __init__(self):
        # 添加 MA5 和 MA10 指标
        self.sma5 = bt.indicators.SimpleMovingAverage(self.data.close, period=3)
        self.sma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=20)
        # 计算 MA5 和 MA10 的交叉信号
        self.crossover = bt.indicators.CrossOver(self.sma5, self.sma10)

    def next(self):
        # 如果 MA5 上穿 MA10，买入
        if self.crossover > 0:
            print("买入")
            self.buy()
        # 如果 MA5 下穿 MA10，卖出
        elif self.crossover < 0:
            print("卖出")
            self.sell()


In [4]:
import backtrader as bt
import pandas as pd
from datetime import datetime

class DynamicPositionStrategy2(bt.Strategy):
    def __init__(self):
        self.sma5 = bt.indicators.SimpleMovingAverage(self.data.close, period=5)
        self.sma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=20)
        self.crossover = bt.indicators.CrossOver(self.sma5, self.sma10)
        # 假设使用波动率指标来动态调整仓位
        self.volatility = bt.indicators.StdDev(self.data.close, period=20)

    def next(self):
        # 获取当前日期
        current_date = self.data.datetime.date(0)
        # 获取当前可用资金
        available_cash = self.broker.get_cash()
        # 获取当前持仓数量
        position_size = self.position.size
        # 获取当前价格
        price = self.data.close[0]

        # 手动计算波动率的均值
        volatility_data = self.volatility.get(size=20)  # 获取最近 20 个周期的波动率数据
        if len(volatility_data) == 20:  # 确保有足够的数据来计算均值
            volatility_mean = sum(volatility_data) / len(volatility_data)
        else:
            # 如果数据不足，使用默认值，这里简单设为 0
            volatility_mean = 0

        if self.crossover > 0:
            # 计算可投资金比例
            if self.volatility[0] > volatility_mean:
                # 高波动率时使用 30% 的资金
                invest_ratio = 0.3
            else:
                # 低波动率时使用 70% 的资金
                invest_ratio = 0.7
            invest_amount = available_cash * invest_ratio

            if invest_amount > 0:
                # 计算可买入数量
                size = invest_amount / price
                self.buy(size=size)
                print(f'{current_date}: 买入信号触发，可用资金: {available_cash:.2f}，投资比例: {invest_ratio * 100:.2f}%，买入数量: {size:.2f}')
            else:
                print(f'{current_date}: 买入信号触发，但可用资金不足')

        elif self.crossover < 0:
            if position_size > 0:
                self.sell(size=position_size)
                print(f'{current_date}: 卖出信号触发，卖出数量: {position_size:.2f}')
            else:
                print(f'{current_date}: 卖出信号触发，但没有持仓')


In [5]:

# 创建 Cerebro 引擎
cerebro = bt.Cerebro()

# 添加策略
cerebro.addstrategy(DynamicPositionStrategy2)

s = datetime.strptime("20241010", "%Y%m%d")
e = datetime.strptime("20250225", "%Y%m%d")
cerebro.adddata(bt.feeds.PandasData(dataname=df,fromdate=s,todate=e))

# 设置初始资金
cerebro.broker.set_cash(1000000.0)

# 设置仓位


# 设置交易手续费
cerebro.broker.setcommission(commission=0.002)

# 运行回测
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('最终资金: %.2f' % cerebro.broker.getvalue())
# 绘制结果
cerebro.plot(style='candlestick', iplot=False, block=True)

Starting Portfolio Value: 1000000.00
2024-11-26: 卖出信号触发，但没有持仓
2024-12-05: 买入信号触发，可用资金: 1000000.00，投资比例: 70.00%，买入数量: 25473.07
2024-12-17: 卖出信号触发，卖出数量: 25473.07
2025-01-24: 买入信号触发，可用资金: 961617.18，投资比例: 70.00%，买入数量: 27942.38
Final Portfolio Value: 967535.93
最终资金: 967535.93


UsageError: Invalid GUI request 'macosx', valid ones are:dict_keys(['inline', 'nbagg', 'webagg', 'notebook', 'ipympl', 'widget', None, 'qt', 'qt5', 'qt6', 'wx', 'tk', 'gtk', 'gtk3', 'osx', 'asyncio'])
